## Felügyelet nélküli telepítő (iso) készítése:

In [ ]:
%%sh
cd ~
rm -f -R ~/ubuntu-iso
pwd
git clone https://github.com/englert/ubuntu-iso.git
cd ~/ubuntu-iso
gnome-terminal -e "sudo ./create-unattended-iso.sh"


## Ubuntu szerver telepítése Virtualboxon:

In [76]:
%%sh
VM="ubu"
RAM="2048"
VRAM="16"
HDD="8192"
LAN="wlp2s0"
ISO="ubuntuserver.iso"
VMs="VirtualBox VMs"
cd ~
VBoxManage createvm --name $VM --ostype "Linux_64" --register
VBoxManage modifyvm $VM --nic1 bridged --bridgeadapter1 $LAN
VBoxManage createhd --filename "$VMs/$VM/$VM.vdi" --size $HDD
VBoxManage storagectl $VM --name "SATA Controller" --add sata
VBoxManage storageattach $VM --storagectl "SATA Controller" --port 0 --device 0 --type hdd --medium "$VMs/$VM/$VM.vdi"
VBoxManage storagectl $VM --name "IDE Controller" --add ide
VBoxManage storageattach $VM --storagectl "IDE Controller" --port 0 --device 0 --type dvddrive --medium $ISO
VBoxManage modifyvm $VM --ioapic on
VBoxManage modifyvm $VM --boot1 dvd --boot2 disk --boot3 none --boot4 none
VBoxManage modifyvm $VM --memory $RAM --vram $VRAM

Virtual machine 'ubu' is created and registered.
UUID: 01c0bb05-3bb7-462a-aaed-e7e58b949f40
Settings file: '/home/sis/VirtualBox VMs/ubu/ubu.vbox'
Medium created. UUID: 7e428c47-a32f-43d0-8b77-a6d00c6b5600


0%...10%...20%...30%...40%...50%...60%...70%...80%...90%...100%


In [77]:
%%sh
VM="ubu"
VBoxManage startvm $VM --type headless

Waiting for VM "ubu" to power on...
VM "ubu" has been successfully started.


In [79]:
%%sh
VBoxManage guestproperty get "ubu" "/VirtualBox VMs/GuestInfo/Net/0/V4/IP"

No value set!


In [103]:
%%sh
cat <<EOF> vboxip.sh 

##!/bin/bash
#
# Translate a MAC address fetched from VirtualBox into a IP address
#

if [  $# -lt 1 ]; then 
  echo -e "\nUsage:\n$0 [virtual machine] \n" 
  exit 1
fi

# Get a string of the form macaddress1=xxxxxxxxxxx
var1=$(VBoxManage showvminfo $1 --machinereadable |grep macaddress1)

# Asdign macaddress1 the MAC address as a value
eval $var1

# assign m the MAC address in lower case
m=$(echo ${macaddress1}|tr '[A-Z]' '[a-z]')

# This is the MAC address formatted with : and 0n translated into n
mymac=$(echo `expr ${m:0:2}`:`expr ${m:2:2}`:`expr ${m:4:2}`:`expr ${m:6:2}`:`expr ${m:8:2}`:`expr ${m:10:2}`)
echo "The MAC address of the virtual machine $1 is $mymac"

# Get known IP and MAC addresses
IFS=$'\n'; for line in $(arp -a); do 
#  echo $line
  IFS=' ' read -a array <<< $line
  ip=$(echo "${array[1]}"|tr "(" " "|tr ")" " ")

  if [ "$mymac" = "${array[3]}" ]; then
    echo "and the IP address is $ip"
  fi

done

XooX
chmod +X vboxip.sh

Oracle VM VirtualBox Command Line Management Interface Version 5.2.6
(C) 2005-2018 Oracle Corporation
All rights reserved.

Usage:

VBoxManage showvminfo       <uuid|vmname> [--details]
                            [--machinereadable]
VBoxManage showvminfo       <uuid|vmname> --log <idx>


Syntax error: VM name or UUID required
sh: 1: Bad substitution
sh: 1: Bad substitution
sh: 1: Bad substitution
sh: 1: Bad substitution
sh: 1: Bad substitution
sh: 1: Bad substitution
sh: 30: Bad substitution
sh: 1: Bad substitution


In [93]:
%%sh
cat <<EOF> mmaa
asdf asdfasd asdf asdf asdf
 asdfasdf asdf asd
    4444444444444444
EOF


In [94]:
ls


kezdetek.ipynb  mmaa  ubuntu-iso/  vboxip.sh


In [105]:
%%sh
./vboxip.sh ubu

The MAC address of the virtual machine ubu is 08:00:27:31:65:22


In [106]:
!arp-scan  08:00:27:31:65:22
                    

You need to be root, or arp-scan must be SUID root, to open a link-layer socket.
link_open: Operation not permitted
